# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [23]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pickle

from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import string
string.punctuation

from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier, MultiOutputRegressor



[nltk_data] Error loading punkt: <urlopen error Tunnel connection
[nltk_data]     failed: 501 Not Implemented>
[nltk_data] Error loading stopwords: <urlopen error Tunnel connection
[nltk_data]     failed: 501 Not Implemented>
[nltk_data] Error loading wordnet: <urlopen error Tunnel connection
[nltk_data]     failed: 501 Not Implemented>


In [24]:
# load data from database
engine = create_engine('sqlite:///UdacityDisasterResponse.db')
df = pd.read_sql_table("Messages_Categories", con=engine)

# df = df[:1000]

# df = pd.read_sql_table("Messages_Categories", con="sqlite:///DisasterResponse.db")
X = df["message"]
Y = df.iloc[:,4:].astype("int")

df.head(5)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [25]:
def tokenize(text):
    messages_tokenized = []
    
    for index, message in enumerate(text):
        message = message.lower()
        
        # Remove punctuation
        # https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
        message = message.translate(str.maketrans('', '', string.punctuation))
        
        # Tokenize into sentences and then words
        sentence_list = sent_tokenize(message)
        
        try:
            words_tokenized = [word_tokenize(sentence) for sentence in sentence_list][0]
        except:
            words_tokenized = message
#             print(Exception)
            print(f"Message {message} at index {index} caused an error. Message not tokenzied.")
            

        # Remove stopwords
        words_tokenized = [w for w in words_tokenized if w not in stopwords.words("english")]
        
        messages_tokenized.append(words_tokenized)
        
    return messages_tokenized


In [26]:
def tokenize_single(text):
    """
    Tokenize text into sentences and then single words. Stopwords are also removed.
    """
    
    message = text.lower()
    
    # Remove punctuation
    # https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
    message = message.translate(str.maketrans('', '', string.punctuation))
        
    # Tokenize into sentences and then words
    sentence_list = sent_tokenize(message)
        
    try:
        words_tokenized = [word_tokenize(sentence) for sentence in sentence_list][0]
    except:
        words_tokenized = [""]
        print(Exception)
        print(f"Text '{text}' replaced with empty string.")

    # Remove stopwords
    words_tokenized = [w for w in words_tokenized if w not in stopwords.words("english")]
    
    
    return words_tokenized


In [27]:
def lemmatize(text):
    """
    This function takes a series of text pieces, tokenizes by words via the tokenize() function and then lemmatizes each token.
    The output is a list of lemmatized tokens per message.
    """
    # tokenize text
    messages_tokenized = tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    messages_lemmatized = []
    
    for message in messages_tokenized:
        try:
            words_lemmatized = [lemmatizer.lemmatize(token) for token in message]
            messages_lemmatized.append(words_lemmatized)
            max_index = messages_tokenized.index(message)
        except:
            print(f"Message at index {max_index + 1} caused an error. Message not lemmatized.")
            

    return messages_lemmatized

In [28]:
def lemmatize_single(text):
    """
    This function takes a piece of text, tokenizes by words via the tokenize() function and then lemmatizes each token.
    The output is a list of lemmatized tokens per corpus.
    """
    # tokenize text
    message_tokenized = tokenize_single(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    words_lemmatized = [lemmatizer.lemmatize(token) for token in message_tokenized]
    
    # Apply porter stemmer
    ps = PorterStemmer()
    words_stemmed = [ps.stem(word) for word in words_lemmatized]
    
    return words_stemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [29]:
from sklearn.pipeline import Pipeline
# from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import hamming_loss

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multioutput import MultiOutputClassifier, MultiOutputRegressor
from sklearn.metrics import classification_report

from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.neural_network import MLPClassifier



In [30]:
Y.sum(axis=1)
categories_colnames = list(Y.columns)
categories_colnames

X_subset = X[:2]
Y_subset = Y.head(2)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
# X_train, X_test, Y_train, Y_test = train_test_split(X_subset, Y_subset)

# Y_train["request"]

print("Done.")

Done.


In [31]:
# # categories_colnames = ["related"] #, "request"]

# # https://towardsdatascience.com/multi-label-text-classification-with-scikit-learn-30714b7819c5

# pipeline = Pipeline([
#         ('vect', CountVectorizer(tokenizer=tokenize_single)),
# #         ('tfidf', TfidfVectorizer(lowercase=False)),
#         ('tfidf', TfidfTransformer()),

#         ('clf', OneVsRestClassifier(MultinomialNB(
#             fit_prior=True, class_prior=None))),
#     ])

# model_dict = {}

# for category in categories_colnames:
#     print(f'... Processing {category}')
#     # train the model using X_dtm & y
#     pipeline.fit(X_train, Y_train[category])
#     # compute the testing accuracy
#     prediction = pipeline.predict(X_test)
#     print(f'Test accuracy is {accuracy_score(Y_test[category], prediction)}')
#     model_dict[category] = pipeline

In [32]:
# print(classification_report(Y_test["related"], prediction))
# print(classification_report(Y_test["related"], model_dict["related"].predict(X_test)))



# Test with MultiOutputClassifier()

In [33]:
pipeline_modular = Pipeline([
        ('vect', CountVectorizer(tokenizer=lemmatize_single)),
#         ('tfidf', TfidfVectorizer(lowercase=False)),
        ('tfidf', TfidfTransformer()),

        # ('clf', MultiOutputClassifier(RandomForestClassifier())),
        # ('clf', MultiOutputClassifier(MultinomialNB(fit_prior=True, class_prior=None))),
        ('clf', MultiOutputClassifier(SGDClassifier())),
        # ('clf', MLPClassifier(random_state=1, max_iter=300)),



    ])

# train the model using X_dtm & y
pipeline_modular.fit(X_train, Y_train)
# compute the testing accuracy
prediction_modular = pipeline_modular.predict(X_test)

c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


<class 'Exception'>
Text '    ' replaced with empty string.


In [34]:
print(classification_report(Y_test, prediction_modular, target_names=categories_colnames))
print(f"Hamming loss: {hamming_loss(Y_test, prediction_modular)}")

                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      4912
               request       0.78      0.57      0.66      1118
                 offer       0.00      0.00      0.00        30
           aid_related       0.75      0.68      0.71      2688
          medical_help       0.66      0.20      0.31       516
      medical_products       0.72      0.22      0.34       338
     search_and_rescue       0.81      0.12      0.21       178
              security       0.00      0.00      0.00       122
              military       0.73      0.17      0.27       215
                 water       0.74      0.65      0.69       404
                  food       0.81      0.71      0.76       739
               shelter       0.82      0.50      0.62       614
              clothing       0.64      0.47      0.55        95
                 money       0.57      0.06      0.11       129
        missing_people       0.00      

c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\q514907\Anaconda3\envs\UdacityProject_2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
print("Test")

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 1

cv = 1

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [54]:
filename_model = "ML_model.sav"

file_model = open(filename_model, "wb")
pickle.dump(pipeline_modular, file_model)
file_model.close()

In [56]:
# import model
file_model = open(filename_model, "rb")
model_imported = pickle.load(file_model)
file_model.close()

In [57]:
prediction_loaded = model_imported.predict(X_test)

print(classification_report(Y_test, prediction_loaded, target_names=categories_colnames))
print(f"Hamming loss: {hamming_loss(Y_test, prediction_loaded)}")

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

# Old Code Pieces

In [ ]:
stop_code  

In [ ]:

pipeline_OvR = Pipeline([
        ('vect', CountVectorizer(tokenizer=lemmatize_single)),
#         ('tfidf', TfidfVectorizer(lowercase=False)),
        ('tfidf', TfidfTransformer()),

        ('clf', OneVsRestClassifier(RandomForestClassifier())),
    ])

# train the model using X_dtm & y
pipeline_OvR.fit(X_train, Y_train)
# compute the testing accuracy
prediction_OvR = pipeline_OvR.predict(X_test)

print(classification_report(Y_test, prediction_OvR))

pipeline_NB = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize_single)),
#         ('tfidf', TfidfVectorizer(lowercase=False)),
        ('tfidf', TfidfTransformer()),

        ('clf', MultiOutputClassifier(MultinomialNB(fit_prior=True, class_prior=None))),
    ])

# train the model using X_dtm & y
pipeline_NB.fit(X_train, Y_train)
# compute the testing accuracy
prediction_NB = pipeline_NB.predict(X_test)

print(classification_report(Y_test, prediction_NB, target_names=categories_colnames))
# SVM
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

pipeline_SVM = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize_single)),
#         ('tfidf', TfidfVectorizer(lowercase=False)),
        ('tfidf', TfidfTransformer()),

        ('clf', MultiOutputClassifier(SGDClassifier())),
    ])

# train the model using X_dtm & y
pipeline_SVM.fit(X_train, Y_train)
# compute the testing accuracy
prediction_SVM = pipeline_SVM.predict(X_test)

print(classification_report(Y_test, prediction_SVM))
hamming_loss(Y_test, prediction_SVM)
# Test Multilayer Perceptron MLP
from sklearn.neural_network import MLPClassifier

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize_single)),
#         ('tfidf', TfidfVectorizer(lowercase=False)),
        ('tfidf', TfidfTransformer()),

        ('clf', MLPClassifier(random_state=1, max_iter=300)),
    ])

# train the model using X_dtm & y
pipeline.fit(X_train, Y_train)
# compute the testing accuracy
prediction = pipeline.predict(X_test)